In [ ]:
# notebook/00_Extraccion_PNOA_OSM_espais-verds.ipynb

# ------------------------------------------------------------------
# 0. IMPORTS Y CONFIG
# ------------------------------------------------------------------

from pathlib import Path
import logging
import requests
import geopandas as gpd
from shapely.geometry import Polygon
import mercantile
import sys
from tqdm import tqdm
from time import sleep
%load_ext autoreload
%autoreload 2

PROJECT_ROOT = Path().resolve().parent
sys.path.insert(0, str(PROJECT_ROOT / 'scripts'))
from valencia_config import VALENCIA

logging.basicConfig(level=logging.INFO, format='%(levelname)s - %(message)s')
logging.info("✅ EE inicializado")

RAW_DIR = PROJECT_ROOT / 'data' / 'raw' / 'pnoa'
PROCESSED_DIR = PROJECT_ROOT / 'data' / 'processed'
RAW_DIR.mkdir(parents=True, exist_ok=True)
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)


def generar_grid_tiles(grid_size: int):
    """
    Devuelve lista de mercantile.Tile centrada en coords y tamaño grid_size×grid_size.
    Para impar: offsets = [-M … 0 … +M]
    Para par:   offsets = [-M … -1, 0 … +M-1]
    """
    lon0, lat0 = VALENCIA['center_wgs'][1], VALENCIA['center_wgs'][0]
    z = VALENCIA['zoom']
    center = mercantile.tile(lon0, lat0, z)

    N = grid_size
    M = N // 2
    if N % 2 == 1:
        offsets = list(range(-M, M+1))
    else:
        offsets = list(range(-M, M))

    tiles = [mercantile.Tile(center.x + dx,
                             center.y + dy,
                             z)
             for dx in offsets for dy in offsets]
    return tiles


def tile_to_polygon(tile):
    """
    Convierte un mercantile.Tile a Polígono WGS84 de sus bounds.
    """
    w, s, e, n = mercantile.bounds(tile.x, tile.y, tile.z)
    return Polygon([(w, s), (w, n), (e, n), (e, s), (w, s)])


def download_tiles_and_build_gdf(grid_size: int, overwrite: bool=False):
    """
    Descarga imágenes aéreas PNOA para una cuadrícula de tamaño dado y 
    genera un GeoDataFrame con su geometría, guardándolo como GeoJSON.
    """
    tiles = generar_grid_tiles(grid_size)
    logging.info(f"🛰️ Descargando {len(tiles)} parches PNOA…")
    records = []
    for idx, t in enumerate(tqdm(tiles, desc="Parches", unit="patch")):
        # Cada 200 peticiones, descansa 5 segundos
        if idx > 0 and idx % 200 == 0:
            logging.info("⏸️ Pausando 5 s para no sobrecargar el servidor")
            sleep(5)

        fn = RAW_DIR / f"valencia_tile_{t.x}_{t.y}_{t.z}.jpg"
        # Si no queremos overwrite y ya existe, saltar
        if not overwrite and fn.exists():
            continue
        url = (
            f"{VALENCIA['pnoa_url']}?"
            "service=WMTS&request=GetTile&version=1.0.0&"
            f"layer={VALENCIA['pnoa_layer']}&format=image/jpeg&"
            f"TileMatrixSet={VALENCIA['tilematrixset']}&"
            f"TileMatrix={t.z}&TileRow={t.y}&TileCol={t.x}"
        )
        try:
            r = requests.get(url)
            r.raise_for_status()
            fn.write_bytes(r.content)
        except Exception as e:
            tqdm.write(f"❌ Error tile {t.x},{t.y},{t.z}: {e}")
        records.append({
            'x': t.x, 'y': t.y, 'z': t.z,
            'geometry': tile_to_polygon(t)
        })
    gdf = gpd.GeoDataFrame(records, crs="EPSG:4326")
    save_path = RAW_DIR.relative_to(PROJECT_ROOT) / "valencia_tiles_grid.geojson"
    gdf.to_file(RAW_DIR / "valencia_tiles_grid.geojson", driver="GeoJSON")
    logging.info(f"📂 GeoJSON guardado en {save_path}")
    return gdf

def get_official_greens() -> gpd.GeoDataFrame:
    """
    Datos oficiales del Ayuntamiento de Valencia en UTM.
    """
    import pyproj
    from shapely.ops import transform
    from shapely.geometry import shape

    limit, start, recs = 1000, 0, []
    url = 'https://valencia.opendatasoft.com/api/records/1.0/search'
    logging.info("🌳 Espais‑verds oficiales…")
    while True:
        params = {
            'dataset': 'espais-verds-espacios-verdes',
            'rows': limit, 'start': start,
            'fields': 'objectid,nombre,barrio,tipologia,geo_shape'
        }
        r = requests.get(url, params=params, timeout=30)
        batch = r.json().get('records', [])
        if not batch: break
        recs.extend(batch); start += limit

    feats = []
    transformer = pyproj.Transformer.from_crs("EPSG:4326", VALENCIA['utm_crs'], always_xy=True)
    for r in recs:
        geom = shape(r['fields']['geo_shape'])
        if geom.is_valid:
            feats.append({
                **{k: r['fields'].get(k) for k in ['objectid','nombre','barrio','tipologia']},
                'geometry': transform(transformer.transform, geom)
            })
    gdf = gpd.GeoDataFrame(feats, crs=VALENCIA['utm_crs'])
    logging.info(f"✅ Oficiales: {len(gdf)} polígonos")
    return gdf

def get_osm_parks() -> gpd.GeoDataFrame:
    """
    Consulta OSM para Valencia y proyecta a UTM.
    """
    import overpy, pyproj
    from shapely.ops import transform

    logging.info("🌳 Parques OSM…")
    api = overpy.Overpass()
    query = f'''
    [out:json][timeout:300];
    area({VALENCIA['osm_area_id']})->.a;
    (
      way["leisure"~"park|garden|recreation_ground"](area.a);
      relation["leisure"~"park|garden|recreation_ground"](area.a);
    );
    (._;>;);
    out geom;
    '''
    res = api.query(query)
    feats = []
    transformer = pyproj.Transformer.from_crs("EPSG:4326", VALENCIA['utm_crs'], always_xy=True)
    for w in res.ways:
        coords = [(float(n.lon), float(n.lat)) for n in w.nodes]
        if len(coords) >= 4:
            poly = Polygon(coords)
            if poly.is_valid:
                feats.append({
                    'nombre': w.tags.get('name', ''),
                    'fuente': 'OSM',
                    'geometry': transform(transformer.transform, poly)
                })
    gdf = gpd.GeoDataFrame(feats, crs=VALENCIA['utm_crs'])
    logging.info(f"✅ OSM: {len(gdf)} polígonos")
    return gdf

# ------------------------------------------------------------------
# 0. PIPELINE
# ------------------------------------------------------------------
if __name__ == "__main__":
    tiles_gdf = download_tiles_and_build_gdf(VALENCIA['grid_size'], overwrite=False)
    official = get_official_greens()
    osm = get_osm_parks()
    verdes = gpd.overlay(official, osm, how='union')
    save_green = PROCESSED_DIR.relative_to(PROJECT_ROOT) / "valencia_zonas_verdes.gpkg"
    verdes.to_file(PROCESSED_DIR / "valencia_zonas_verdes.gpkg", driver="GPKG")
    logging.info(f"📂 GeoPackage guardado en {save_green} con {len(verdes)} polígonos")


INFO - ✅ EE inicializado
INFO - 🛰️ Descargando 225 parches PNOA…
Parches: 100%|██████████| 225/225 [00:28<00:00,  7.89patch/s]
INFO - Created 225 records
INFO - 📂 GeoJSON guardado en data\raw\pnoa\valencia_tiles_grid.geojson
INFO - 🌳 Espais‑verds oficiales…
INFO - ✅ Oficiales: 373 polígonos
INFO - 🌳 Parques OSM…
INFO - ✅ OSM: 1715 polígonos
INFO - Created 2,513 records
INFO - 📂 GeoPackage guardado en data\processed\valencia_zonas_verdes.gpkg con 2513 polígonos
